In [66]:
import sqlite3
import pandas as pd

Hello! For Aneeza, Caegs, Rae, and Raymond, this is to combine all of the datafiles into one database. We have two tables in ```fires.db```.

# Filtering Down the US-Wildfires

The columns that we are going to use are:
* FOD_ID
    * Global unique identifier.
* DISCOVERY_DATE 
    * Date on which the fire was discovered or confirmed to exist. Note that this is in the Julian calendar.
* DISCOVERY_TIME
    * Time of day that the fire was discovered or confirmed to exist.
* CONT_DOY
    * Day of year on which the fire was declared contained or otherwise controlled. Note that this is in the Julian calendar.
* CONT_TIME
    * Time of day that the fire was declared contained or otherwise controlled (hhmm where hh=hour, mm=minutes).
* FIRE_SIZE
    * Estimate of acres within the final perimeter of the fire.
* STATE
    * Two-letter alphabetic code for the state in which the fire burned (or originated), based on the nominal designation in the fire report.



In [ ]:
# Paths to databases
source_db = '../mnt/data/public/us-wildfires/FPA_FOD_20170508.sqlite'

# Connect to source and destination databases
source_conn = sqlite3.connect(source_db)
dest_conn = sqlite3.connect('fires.db')

source_cursor = source_conn.cursor()
dest_cursor = dest_conn.cursor()

# Create table in destination database (modify as needed)
create_table_query = """
CREATE TABLE IF NOT EXISTS us_fires (
    FOD_ID INT UNIQUE,
    DISCOVERY_DATE REAL,
    DISCOVERY_TIME TEXT,
    CONT_DATE REAL,
    CONT_TIME TEXT,
    FIRE_SIZE REAL,
    STATE TEXT
)
"""
dest_cursor.execute(create_table_query)

# Copy data from source to destination
query = """
SELECT FOD_ID, DISCOVERY_DATE, DISCOVERY_TIME, CONT_DATE, CONT_TIME, FIRE_SIZE, STATE
FROM Fires
"""

source_cursor.execute(query)
rows = source_cursor.fetchall()

# Commit and close connections
dest_conn.commit()
source_conn.close()

# Just for information
print("Below is the table info for us_fires:")
for row in dest_cursor.execute('PRAGMA table_info("us_fires")').fetchall():
    print(row)
    
dest_conn.close()

Below is the table info for us_fires:
(0, 'FOD_ID', 'INT', 0, None, 0)
(1, 'DISCOVERY_DATE', 'REAL', 0, None, 0)
(2, 'DISCOVERY_TIME', 'TEXT', 0, None, 0)
(3, 'CONT_DATE', 'REAL', 0, None, 0)
(4, 'CONT_TIME', 'TEXT', 0, None, 0)
(5, 'FIRE_SIZE', 'REAL', 0, None, 0)
(6, 'STATE', 'TEXT', 0, None, 0)


# Filtering Down the London Fire Brigade

For the London Fire Brigade, there is the Incident data and the Mobilisation data. For the Incident data, we will be looking at the columns:
* IncidentNumber
* DateOfCall
* IncidentGroup	
* StopCodeDescription	
* SpecialServiceType
* Notional Cost (£)


In [68]:
# Loading Incident Data Files

# We will get only from 2009 to 2022
incident_2009_2022 = '../mnt/data/public/lfb/LFB Incident data - Datastore - with notional cost and UPRN from January 2009.csv'
incident_2009_2022 = pd.read_csv(incident_2009_2022, dtype={"235138081": str})
features = ['235138081', '01 Jan 2009', 'Special Service', 'Special Service.1', 'RTC', '255']
incident_2009_2022 = incident_2009_2022[features]
incident_2009_2022.columns = ['IncidentNumber', 'DateOfCall', 'IncidentGroup', 'StopCodeDescription', 'SpecialServiceType', 'Notional Cost (£)']

# Get only dates from 2023
incident_2020_2023 = '../mnt/data/public/lfb/LFB Incident data from January 2020 to August 2023.xlsx'
incident_2020_2023 = pd.read_excel(incident_2020_2023, usecols=['IncidentNumber', 'DateOfCall', 'IncidentGroup', 'StopCodeDescription', 'SpecialServiceType', 'Notional Cost (£)'])

Further filter out data, and combine the two dataframes into one big dataframe.

In [69]:
# Create a new dataframe from incident_2020_2023 where it only has Fires from 2023
filt_year = incident_2020_2023['DateOfCall'].dt.year == 2023
incident_2023 = incident_2020_2023[filt_year]
all_incidents = pd.concat([incident_2009_2022, incident_2023])

# Convert DateOfCall to proper timestamp
all_incidents["DateOfCall"] = pd.to_datetime(all_incidents["DateOfCall"], format="%d %b %Y")


For the Mobilisation data, we will be looking at the columns:
* IncidentNumber
* DateAndTimeMobilised	
* DateAndTimeMobile	
* DateAndTimeArrived	
* DateAndTimeLeft
* DelayCode_Description

In [70]:
# Loading Mobilisation Data Files
all_mobilisation = '../mnt/data/public/lfb/mobilisation/LFB Mobilisation data from January 2009 to August 2023.csv'
all_mobilisation = pd.read_csv(all_mobilisation, dtype={"235138081": str, '01/01/2009 00:23:55':str})

# Cleaning the files
features = [all_mobilisation.columns[i] for i in [0, 6, 7, 8, 12, 21]]
all_mobilisation = all_mobilisation[features]
all_mobilisation.columns = ['IncidentNumber', 'DateAndTimeMobilised', 'DateAndTimeMobile', 'DateAndTimeArrived', 'DateAndTimeLeft', 'DelayCode_Description']

# Converting the datetime columns into an actual datetime format
datetime_columns = ["DateAndTimeMobilised", "DateAndTimeMobile", "DateAndTimeArrived", "DateAndTimeLeft"]
all_mobilisation[datetime_columns] = all_mobilisation[datetime_columns].apply(
    pd.to_datetime, format="%d/%m/%Y %H:%M:%S", errors="coerce")

In [ ]:
# Converting the pandas dataframe to a SQL database
conn = sqlite3.connect('london_fires.db')
all_incidents.to_sql('london_incidents', conn, if_exists='replace', index=False)
all_mobilisation.to_sql('london_mobilisation', conn, if_exists='replace', index=False)


2243558

We will now join the ```london_incidents``` and ```london_mobilisation``` using the key-id of IncidentNumber.

In [ ]:
# Read the query and also save it into fires.db
query = """
SELECT *
FROM london_incidents
JOIN london_mobilisation
USING (IncidentNumber)
"""

london_fires = pd.read_sql(query, conn)


Below is the table info for london_fires:
